<a href="https://colab.research.google.com/github/vamsikrishna00466/time_series/blob/main/Multi%20variate%20time%20series%20by%20VAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing pandas and warnings
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#Importing data
df=pd.read_excel('/content/drive/MyDrive/SCA Projects/Forecasting /Raw Data for Forecasting Project.xlsx')

##Initial preprocessing(12-12-2021) work-Function
def data_preprocess1(df):
  df = df.drop(df.columns.to_series()["Unnamed: 6":"Receipt Details"], axis=1)
  df.columns = df.iloc[0]
  df = df[1:].reset_index(drop=bool)
  del df['Total']
  df = df.melt(id_vars=["Part No.", "Unit Price",'Category','Sales MOQ','Opening Stock'],
               var_name="Month",
               value_name="demand")
  df['Month']=pd.to_datetime(df['Month']).dt.to_period('M')
  df['Unit Price']=df['Unit Price'].astype('float')
  df['Sales MOQ']=df['Sales MOQ'].astype('float')
  df['Opening Stock']=df['Opening Stock'].astype('float')
  df['demand']=df['demand'].astype('float')
  return df

##Initial preprocessing raw data
df1 = data_preprocess1(df)

##Basic Analysis Function for initial preprocessed data
def basic_analysis(df):
  print('*****Initial Information about Data*****\n')
  print(df.head())
  print('\nNumber of Rows and Columns:',df.shape,'\n--------------------')
  Null = df.isnull().sum()
  Null = Null[Null>0]
  print('missing data:\n',Null)
  print('--------------------\nData Types:\n',df.dtypes)
  print('--------------------\nCategorical variables:',df.select_dtypes(include=['object']).columns.to_list(),'And Number of columns:',len(df.select_dtypes(include=['object']).columns.to_list()))
  print('--------------------\nNumerical variables:',df.select_dtypes(include=[int,float]).columns.to_list(),'And Number of columns:',len(df.select_dtypes(include=[int,float]).columns.to_list()))
  print('--------------------\nDate time variables:',df.select_dtypes(include=['period[M]']).columns.to_list())
  print('--------------------\n')
  print(round(df.describe()).astype(int))
  print('--------------------\n')
  print(df.describe(include=object))
  print('--------------------\n')
  print(df['Month'].describe())
  print('--------------------\n')

#Basic information about initial pre processed data
basic_analysis(df1)

#group by month and categorya and aggregation with demand sum
df2 = df1.groupby(['Category','Month'])['demand'].sum().reset_index()
print('Group by ccompleted')

import matplotlib.pyplot as plt
import seaborn as sns
sns.catplot(x="Month", y="demand", hue='Category', data=df2, kind='point',height=5)
plt.show()

##converting categories into columns
df3 = df2.pivot(index='Month', columns='Category', values='demand')

print('Rows and columns of categories columns month wise demand:',df3.shape)

##sort data by date(ascending order)
df3.sort_index(inplace=True)

df3 = round(df3).astype(int)

print(round(df3.describe()).astype(int))

## Hyphothesis Testing for distribution checking(normally distributed or not)
alpha = 0.5
H0 = 'Data is normal'
Ha = 'Data is not normal'
from scipy.stats import shapiro
for i in df3.columns.to_list():
  p = round(shapiro(df3[i])[1], 2)
  print(f'**************{i}**************')
  if p > alpha:
    print(f"{p} > {alpha}. We fail to reject Null Hypothesis. {H0}")
  else:
    print(f"{p} <= {alpha}. We reject Null Hypothesis. {Ha}")
  print('--------------\n')

##ploting line chart of categoris with respect to demand
import matplotlib.pyplot as plt
%matplotlib inline
for i in df3.columns.to_list():
  df3[i].plot(label=i)
  plt.suptitle(i)
  plt.show()
  print('\n')

##grangercausalitytests test
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df3, variables = df3.columns)        


##coint_johansen test
from statsmodels.tsa.vector_ar.vecm import coint_johansen
def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(df3)

##Stationary test
from statsmodels.tsa.stattools import adfuller
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.") 

# ADF Test on each column
for name, column in df3.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

# 1st difference for make data stationary
df3_differenced = df3.diff().dropna()

# ADF Test on each column again
for name, column in df3_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

#creating the train and validation set
train = df3[:int(0.8*(len(df3)))]
valid = df3[int(0.8*(len(df3))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

#mean squred error
from sklearn.metrics import mean_squared_error
print('Mean squre Error:',round(np.sqrt(mean_squared_error(valid, prediction, squared=False))))
